# 1. Load SQLite and Database
- 'CourseData.db'
- note the 3 backslashes

In [2]:
%reload_ext sql
%sql sqlite:///CourseData.db

'Connected: @CourseData.db'

In [18]:
%%sql
DROP TABLE IF EXISTS CourseData;

 * sqlite:///CourseData.db
Done.


[]

- safety precautions

In [19]:
%%sql
sqlite:///CourseData.db
DROP TABLE IF EXISTS COURSECATALOGS;
DROP TABLE IF EXISTS COURSES;
DROP TABLE IF EXISTS TEACHERS;
DROP TABLE IF EXISTS COURSEMEETINGS;

Done.
Done.
Done.
Done.


[]

# 2. Do COUNT statements for 3 .csv Files
- ensure .csv name spelling is exact

In [9]:
%%sql
SELECT Count(*) FROM import_course_meetings;

 * sqlite:///CourseData.db
Done.


Count(*)
317339


In [10]:
%%sql
SELECT Count(*) FROM import_courses;

 * sqlite:///CourseData.db
Done.


Count(*)
15955


# 3. Create Tables
- primary key is generated on every table by naming new surrogate key/field that isn't already in .csv files.
- don't need NOT NULL for PK because it's implied in SQLite.


In [20]:
%%sql
CREATE TABLE COURSECATALOGS (
    CCID INTEGER PRIMARY KEY,
    catalog_id TEXT,
    program_code TEXT,    
    program_name TEXT,
    course_title TEXT,
    prereqs TEXT,    
    coreqs TEXT,
    fees TEXT,
    attributes TEXT,    
    description TEXT
);

 * sqlite:///CourseData.db
Done.


[]

In [27]:
%%sql
DROP TABLE IF EXISTS COURSES;

 * sqlite:///CourseData.db
Done.


[]

In [28]:
%%sql
CREATE TABLE COURSES (
                 CID INTEGER PRIMARY KEY,
                 crn INTEGER,
                 term TEXT,
                 section TEXT,
                 credits TEXT,
                 title TEXT,
                 meetings TEXT,
                 timecodes TEXT,
                 primary_instructor TEXT,
                 cap TEXT,
                 act TEXT,
                 rem TEXT,
                 CCID INTEGER,
                 FOREIGN KEY(CCID) REFERENCES COURSECATALOGS(CCID)
);


 * sqlite:///CourseData.db
Done.


[]

In [23]:
%%sql
CREATE TABLE TEACHERS (
 TID INTEGER PRIMARY KEY,
 primary_instructor TEXT NOT NULL,
 CID INTEGER,
 FOREIGN KEY (CID) REFERENCES COURSES(CID)
);


 * sqlite:///CourseData.db
(sqlite3.OperationalError) table TEACHERS already exists [SQL: 'CREATE TABLE TEACHERS (\n TID INTEGER PRIMARY KEY,\n primary_instructor TEXT NOT NULL,\n CID INTEGER,\n FOREIGN KEY (CID) REFERENCES COURSES(CID)\n);'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [24]:
%%sql
CREATE TABLE COURSEMEETINGS (
    CMID INTEGER PRIMARY KEY,
    location TEXT NOT NULL,	
    day TEXT NOT NULL,
    start TEXT NOT NULL,
    end TEXT NOT NULL,
    CID INTEGER,
    TID INTEGER,
    FOREIGN KEY (CID) REFERENCES COURSES(CID),
    FOREIGN KEY (TID) REFERENCES TEACHERS(TID)
);

 * sqlite:///CourseData.db
Done.


[]

# 4. Do INSERT Statements
- do not include generated PKs
- *reasoning* - PKs aren't included in the imported tables, so we can't insert them from there.
- implied (since all PK/FKs are generated) that  PK/FKs are distinct.

In [25]:
%%sql
INSERT INTO COURSECATALOGS (
    catalog_id,
    program_code,  
    program_name,
    course_title,
    prereqs,    
    coreqs,
    fees,
    attributes,    
    description) 
SELECT DISTINCT 
    catalog_id,
    program_code,  
    program_name,
    course_title,
    prereqs,    
    coreqs,
    fees,
    attributes,    
    description
    FROM import_Course_Catalog;

 * sqlite:///CourseData.db
2221 rows affected.


[]

In [1]:
%%sql
INSERT INTO COURSES (
                 crn,
                 term,
                 section,
                 credits,
                 title,
                 meetings,
                 timecodes,
                 primary_instructor,
                 cap,
                 act,
                 rem,
)
 SELECT DISTINCT
                 crn,
                 term,
                 section,
                 credits,
                 title,
                 meetings,
                 timecodes,
                 primary_instructor,
                 cap,
                 act,
                 rem
                FROM import_courses
                JOING COURSEMEETINGS (COURSEMEETINGS.CCID=COURSES.CCID);

UsageError: Cell magic `%%sql` not found.


In [32]:
%%sql INSERT INTO COURSEMEETINGS (
    location,	
    day,
    start,
    end
)
 SELECT DISTINCT
    location,	
    day,
    start,
    end
    FROM import_course_meetings;
    


 * sqlite:///CourseData.db
296908 rows affected.


[]

In [34]:
%%sql INSERT INTO TEACHERS (
    primary_instructor
)
 SELECT DISTINCT
    primary_instructor
    FROM import_courses;

 * sqlite:///CourseData.db
1105 rows affected.


[]

# 5. Run SELECT Queries

In [1]:
%reload_ext sql
%sql sqlite:///CourseData.db

'Connected: @CourseData.db'

## SELECT Query for all unique classrooms 
- only included those with a character lenght of 7 (some had 2, 8, etc.)
- can't figure out how to get a distinct location when including additinonal fields.

In [37]:
%%sql
SELECT DISTINCT 
location
FROM COURSEMEETINGS
WHERE LENGTH(location) = 7
ORDER BY location
LIMIT 20
;

 * sqlite:///CourseData.db
Done.


location
BCC 200
BLM 112
BNW 124
BNW 127
BNW 128
BNW 130
BNW 131
BNW 133
BNW 136
BNW 137


## SELECT Query displaying all courses in MSBA Program '18-'19
- still have to get rid of blank by making defining description as NOT NULL.
- is there an easier way?

In [35]:
%%sql
SELECT course_title AS Course, program_name AS Program, catalog_id AS Code, description AS Description
FROM COURSECATALOGS
WHERE program_name = 'Information Systems'
AND
Code LIKE "IS 05%"
ORDER BY program_name
LIMIT 50;

 * sqlite:///CourseData.db
Done.


Course,Program,Code,Description
Information Systems and Database Management,Information Systems,IS 0500,"This course introduces the basic concepts and tools relevant to information systems and database management, and their enabling roles in business strategies and operations. Case studies are used to facilitate discussions of practical applications and issues involving strategic alignments of organizations, resource allocation, integration, planning, and analysis of cost, benefit and performance in light of the big data challenges. Specific emphases involve database design and implementation and emerging strategies and technologies such as business intelligence, big data management, web security, and online business analytics."
International Information Systems,Information Systems,IS 0501,"This course examines information technology environments around the world, and attendant challenges to business strategy and information systems design. The course identifies geographic and institutional variables that create borders in the global Internet economy: material infrastructures, socio-economic elements, and political-legal systems. The course emphasizes national and regional strategies, emergent technologies, hybrid systems, and equity issues."
Python for Business Analytics,Information Systems,IS 0505,"In this course, we introduce Python as a language and tool for collecting, preprocessing, and visualizing data for business analytics. since Python is one of the most popular programming languages, along with R, in data mining and business analytics, its fundamental programming logic and knowledge is essential for students to apply in data mining and to succeed in the job market. Specifically, this course focuses on the data-engineering phase, which includes collecting, preprocessing, and visualizing data, with respect to applications in business modeling, optimization, and statistical analysis. In addition, a number of mini projects will be used as vehicles to cover the main applications of data analytics, including recommender systems, text analytics, and web analytics."
Databases for Business Analytics,Information Systems,IS 0510,"This course introduces databases and data management in three parts. The first part covers basic database fundamentals. The second part is a hands-on introduction to Structured Query Language (SQL) for defining, manipulating, accessing, and managing data, accompanied by the basics of data modeling and normalization needed to ensure data integrity. The course concludes with a comprehensive database project that gives each student the opportunity to integrate and apply the new knowledge and skills learned from this class. Advanced topics such as distributed database systems, data services, and NoSQL databases are also discussed."
Project Management,Information Systems,IS 0520,"This course explores the process and practice of project management. Topics to be covered include project lifecycle and organizations, teambuilding and productivity, task scheduling and resource allocation, and progress tracking and control. Cases will be used to consider the implications for change management, consulting, IT implementation, and other related disciplines. Small team projects and experiential exercises will also be used to provide an active learning environment. This course is designed to count toward professional project management certification."
Data Mining and Business Intelligence,Information Systems,IS 0540,"This course will change the way you think about data and its role in business. Businesses, governments, and individuals create massive collections of data as a byproduct of their activity. Increasingly, managers rely on intelligent technology to systematically analyze data to improve their decision-making. In many cases, automating analytical and decision-making processes is necessary because of the large volume of data and the speed with which new data are generated. In this course, we will examine how d

In [46]:
%%sql
SELECT
t.primary_instructor, c.title
FROM TEACHERS AS t
JOIN COURSES AS c ON c.CID = t.CID
;

 * sqlite:///CourseData.db
Done.


primary_instructor,title
